In [3]:
import pandas as pd
from unidecode import unidecode

# Load the CSV files
tm_values = pd.read_csv('../data/tm/processed/tm_values.csv')
player_stats = pd.read_csv('../data/fbref/engineered/outfield-goalkeeper-combined/fbref__outfield_player_goalkeeper_stats_combined_latest.csv')
mapping_data = pd.read_csv('../data/mapping/fbref_to_tm_mapping.csv', encoding='ISO-8859-1')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [4]:
# Delete entries from season 2021/2022 from fbref data
player_stats = player_stats[player_stats['Season'] != '2021-2022']

# Filter the DataFrame by position
gk_stats = player_stats[player_stats['Pos'] == 'GK']
out_stats = player_stats[player_stats['Pos'] != 'GK']

In [5]:
# Function to consolidate duplicate rows for each player in each season
def consolidate_season_data(player_data):
    # Identify all unique seasons for the player
    unique_seasons = player_data['Season'].unique()

    # Create an empty DataFrame to store the consolidated data
    consolidated_data = pd.DataFrame()

    for season in unique_seasons:
        # Filter the data for the current season
        season_data = player_data[player_data['Season'] == season]

        # Group the data by club
        clubs = season_data['Squad'].unique()

        for club in clubs:
            # Filter the data for the current club
            club_data = season_data[season_data['Squad'] == club]

            if len(club_data) == 2:  # There should be two duplicated rows
                # Select the relevant parts of each row
                part_1 = club_data.iloc[0, :division_index]  # Data up to the 'GA' column
                part_2 = club_data.iloc[1, division_index:]  # Data from the 'GA' column onwards

                # Combine the two parts into a single row
                combined_row = pd.concat([part_1, part_2])
                # Append the combined row to the consolidated DataFrame
                consolidated_data = pd.concat([consolidated_data, combined_row.to_frame().T], ignore_index=True)
            else:
                # If there is no duplication (or some other unexpected case), keep the data as is
                consolidated_data = pd.concat([consolidated_data, club_data], ignore_index=True)

    return consolidated_data

# Identify the index of the 'GA' column
division_column = 'GA'
division_index = gk_stats.columns.get_loc(division_column)

# Apply the function to consolidate data for each player
gk_stats = gk_stats.groupby(['Player', 'Born']).apply(consolidate_season_data).reset_index(drop=True)

/tmp/ipykernel_30618/2585269126.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gk_stats = gk_stats.groupby(['Player', 'Born']).apply(consolidate_season_data).reset_index(drop=True)


In [6]:
# Get the list of all columns
all_columns = player_stats.columns.tolist()

# Find the index of the columns for goalkeepers and outfield players
start_index_out = all_columns.index('Gls') # +1 to exclude 'Matches'
end_index_out = all_columns.index('Position Grouped') + 1  # +1 to include 'League Name'

# Select the desired columns for goalkeepers
# Select the desired columns for outfield players
out_columns = ['Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', 'MP', 'Starts', 'Min', '90s'] + all_columns[start_index_out:end_index_out]

# Filter the DataFrame for outfield players and select the desired columns
out_stats = out_stats[out_columns]

In [7]:
def process_player_stats(df, num_seasons=4, min_minutes=350):
    # Create a copy of the DataFrame
    df = df.copy()

    # Step 0: Normalize the Season format from '2017-2018' to '2017/2018'
    df['Season'] = df['Season'].apply(lambda x: x.replace('-', '/') if isinstance(x, str) else x)

    # Create a composite key to uniquely identify each player
    df['Composite Key'] = df['Player'] + df['Born'].astype(str)

    # Step 1: Filter to keep the row with more minutes for the same season
    df = df.sort_values('Min', ascending=False).drop_duplicates(['Season', 'Composite Key'], keep='first')

    # Step 2: Ensure the player has data for all seasons
    season_counts = df['Composite Key'].value_counts()
    df = df[df['Composite Key'].isin(season_counts[season_counts >= num_seasons].index)]

    # Step 3: Ensure the player has at least one season with 350+ minutes
    def has_high_minutes(group):
        return any(group['Min'] >= min_minutes)
    df = df.groupby('Composite Key').filter(has_high_minutes)

    # Step 4: Ensure Age follows a sequential order
    # Sort by Player and Season
    df.sort_values(['Player', 'Season'], inplace=True)
    df['Age_diff'] = df.groupby('Composite Key')['Age'].diff()
    non_sequential = df[(df['Age_diff'] != 1) & (~df['Age_diff'].isnull())]['Composite Key'].unique()
    df = df[~df['Composite Key'].isin(non_sequential)]

    # Step 5: Validate players' Born year consistency
    valid_born = df.groupby('Composite Key')['Born'].nunique()
    valid_born = valid_born[valid_born == 1].index
    df = df[df['Composite Key'].isin(valid_born)]

    # Step 6: Fix encoding issues in Player and Squad names
    def fix_encoding_issues(name):
        try:
            return name.encode('latin1').decode('utf-8')
        except (UnicodeEncodeError, UnicodeDecodeError):
            return name

    df['Player'] = df['Player'].apply(fix_encoding_issues)
    df['Squad'] = df['Squad'].apply(fix_encoding_issues)

    # Drop the Composite Key and Age_diff columns as they are no longer needed
    df = df.drop(columns=['Composite Key', 'Age_diff'])

    return df

# Apply the integrated function to both goalkeepers and outfield players
filtered_gk_stats = process_player_stats(gk_stats)
filtered_out_stats = process_player_stats(out_stats)

In [8]:
# Update 'Player' name for 'Marcelo' born in 1988 to 'Marcelo Junior'
filtered_out_stats.loc[(filtered_out_stats['Player'] == 'Marcelo') &
                       (filtered_out_stats['Born'] == 1988.0), 'Player'] = 'Marcelo Júnior'

# Update 'Player' name for 'Raul Garcia' born in 1989 to 'Raul Carnero'
filtered_out_stats.loc[(filtered_out_stats['Player'] == 'Raúl García') &
                       (filtered_out_stats['Born'] == 1989.0), 'Player'] = 'Raúl Carnero'

# Update 'Player' name for José Luis Gayà
filtered_out_stats.loc[(filtered_out_stats['Player'] == 'JosÃ© Luis GayÃ'), 'Player'] = 'José Luis Gayà'

In [9]:
import pandas as pd

# Convert date column to datetime
tm_values['date'] = pd.to_datetime(tm_values['date'])

# Function to determine season based on date
def get_season(date):
    year = date.year
    month = date.month
    if year == 2017 and month in [5, 6]:
        return '2017/2018'
    elif month >= 7:
        return f"{year}/{year+1}"
    else:
        return f"{year-1}/{year}"

# Apply the season function
tm_values['season'] = tm_values['date'].apply(get_season)

# Sort by player_id, season, and date
tm_values_sorted = tm_values.sort_values(['player_id', 'season', 'date']).reset_index(drop=True)

# Function to get the first and last market values for each season, preserving player_name and club
def select_market_values(df):
    first_value = df.iloc[0]['market_value']
    last_value = df.iloc[-1]['market_value']
    position = df.iloc[0]['position']
    player_name = df.iloc[0]['player_name']  # Preserve player_name
    club = df.iloc[-1]['club']  # Keep the club corresponding to the last market value
    return pd.Series({
        'player_name': player_name,
        'club': club,
        'market_value_1': first_value,
        'market_value_2': last_value,
        'position': position
    })

# Apply the function to group by player_id and season
season_values = tm_values_sorted.groupby(['player_id', 'season']).apply(select_market_values).reset_index()

# Manually update the 2020/2021 season with the first value from 2021/2022
for idx in range(len(season_values) - 1):
    if season_values.at[idx, 'season'] == '2020/2021':
        next_season_value = season_values.at[idx + 1, 'market_value_1']
        if season_values.at[idx + 1, 'season'] == '2021/2022':
            season_values.at[idx, 'market_value_2'] = next_season_value

# Ensure that the 2021/2022 season is completely excluded
filtered_season_values = season_values[season_values['season'] != '2021/2022']

/tmp/ipykernel_30618/1480683103.py:39: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  season_values = tm_values_sorted.groupby(['player_id', 'season']).apply(select_market_values).reset_index()


In [10]:
def fix_encoding_issues(name):
        try:
            return name.encode('latin1').decode('utf-8')
        except (UnicodeEncodeError, UnicodeDecodeError):
            return name

mapping_data['PlayerFBref'] = mapping_data['PlayerFBref'].apply(fix_encoding_issues)

In [11]:
# Function to extract player_id
def extract_player_id(url):
    try:
        # Attempt to split and convert to integer
        return int(url.split('/')[-1])
    except ValueError:
        # Return None if it fails
        return None

# Apply the function to extract player_id
mapping_data['player_id'] = mapping_data['UrlTmarkt'].apply(extract_player_id)

# Drop rows where player_id is None
mapping_data = mapping_data.dropna(subset=['player_id'])

# Convert player_id to integer type
mapping_data['player_id'] = mapping_data['player_id'].astype(int)

In [12]:
ids_to_exclude = [229736, 627231, 291968, 646658, 385271, 565232, 257097, 479647, 519731, 808509, 738498, 870414, 704558,
                  627230, 75081, 442818, 471474, 227085, 232142, 155470, 461906, 626724, 235568, 342024, 746908, 250144,
                  573907, 277118, 910216, 697818, 54155, 538234, 406675, 443721, 516722, 668268, 54132, 582301, 127554,
                  231289, 817665, 848479, 148765, 432895, 428787, 531916, 498235, 33947, 52894, 1053934, 187099, 170385,
                  820227, 117549, 630995, 743593, 631785, 175256, 1012158, 583607, 153207, 55338, 252900]

ids_to_name = {44501: 'Marcelo Júnior', 139434: 'Raúl Carnero'}

# Filter out the players to exclude
mapping_data = mapping_data[~mapping_data['player_id'].isin(ids_to_exclude)]

# Update player names based on the player_id using the map function
mapping_data['PlayerFBref'] = mapping_data['player_id'].map(ids_to_name).fillna(mapping_data['PlayerFBref'])

In [13]:
# Merge filtered_out_stats with mapping_data to get player_id
merged_out_stats_with_ids = pd.merge(filtered_out_stats, mapping_data[['PlayerFBref', 'player_id']],
                                     left_on='Player', right_on='PlayerFBref', how='left')

# Merge the outfield stats with the market value data
final_merged_out_stats = pd.merge(merged_out_stats_with_ids, filtered_season_values[['market_value_1', 'market_value_2', 'season', 'player_id', 'position']],
                                  left_on=['Season', 'player_id'], right_on=['season', 'player_id'], how='left')

# Clean up unnecessary columns
final_merged_out_stats = final_merged_out_stats.drop(columns=['PlayerFBref', 'season'])

# Merge filtered_gk_stats with mapping_data to get player_id
merged_gk_stats_with_ids = pd.merge(filtered_gk_stats, mapping_data[['PlayerFBref', 'player_id']],
                                    left_on='Player', right_on='PlayerFBref', how='left')

# Merge the goalkeeper stats with the market value data
final_merged_gk_stats = pd.merge(merged_gk_stats_with_ids, filtered_season_values[['market_value_1', 'market_value_2', 'season', 'player_id', 'position']],
                                  left_on=['Season', 'player_id'], right_on=['season', 'player_id'], how='left')

# Clean up unnecessary columns
final_merged_gk_stats = final_merged_gk_stats.drop(columns=['PlayerFBref', 'season'])

# Filter out players who have any NaN values in market_value_1 or market_value_2
players_with_nan_values = final_merged_out_stats[final_merged_out_stats[['market_value_1', 'market_value_2']].isna().any(axis=1)]['Player'].unique()

# Remove all records of these players from the final_merged_out_stats DataFrame
final_merged_out_stats = final_merged_out_stats[~final_merged_out_stats['Player'].isin(players_with_nan_values)]

In [14]:
position_map = {
    'Defender - Centre-Back': 'Centre-Back',
    'Defender - Left-Back': 'Wing-Back',
    'Defender - Right-Back': 'Wing-Back',
    'midfield - Defensive Midfield': 'Defensive Midfielder',
    'midfield - Central Midfield': 'Central Midfielder',
    'midfield - Attacking Midfield': 'Attacking Midfielder',
    'attack - Second Striker': 'Attacking Midfielder',
    'midfield - Left Midfield': 'Winger',
    'midfield - Right Midfield': 'Winger',
    'attack - Left Winger': 'Winger',
    'attack - Right Winger': 'Winger',
    'attack - Centre-Forward': 'Centre-Forward'
}

position_missing_map = {
    'Alessio Romagnoli': 'Centre-Back',
    'Carlos Bacca': 'Centre-Forward',
    'Dani Carvajal': 'Wing-Back',
    'Facundo Roncaglia': 'Centre-Back',
    'José Campaña': 'Central Midfielder',
    'José Ángel': 'Wing-Back',
    'Marcelo Júnior': 'Wing-Back',
    'Mário Rui': 'Wing-Back',
    'Rúben Vezo': 'Centre-Back',
    'Sebastien De Maio': 'Centre-Back'
}

# Apply the mapping to the 'position' column
final_merged_out_stats['Position'] = final_merged_out_stats['position'].map(position_map)
final_merged_out_stats['Position'] = final_merged_out_stats['Player'].map(position_missing_map).fillna(final_merged_out_stats['Position'])

In [15]:
# Create a copy of the DataFrame
final_out_stats_filtered = final_merged_out_stats.copy()

# Renaming columns
final_out_stats_filtered.rename(columns={
    'player_id': 'ID',
    'Squad': 'Team',
    'Nationality Cleaned': 'Nationality',
    'market_value_1': 'MV1',
    'market_value_2': 'MV2',
    'G-PK': 'npG',
    'G-PK.1': 'npG/90',
    'np:G-xG' : 'npG-npxG',
    'Cmp': 'PassCmp',
    'Att': 'PassAtt',
    'Cmp%': 'Pass%',
    'Prog': 'PrgPass',
    'Prog.1': 'PrgPassRec',
    'SCA90': 'SCA/90',
    'GCA90': 'GCA/90',
    'Succ': 'PressSucc',
    '%': 'Press%',
    'Succ%': 'Drib%',
    '#Pl': 'DribSucc',
    'Won': 'ADuelWon',
    'Won%': 'ADuel%'},
                                inplace=True)

# Creating new columns using .loc
final_out_stats_filtered.loc[:, 'DribSucc/90'] = (final_out_stats_filtered['DribSucc'] / final_out_stats_filtered['90s']).round(2)
final_out_stats_filtered.loc[:, 'PassCmp/90'] = (final_out_stats_filtered['PassCmp'] / final_out_stats_filtered['90s']).round(2)
final_out_stats_filtered.loc[:, 'PassAtt/90'] = (final_out_stats_filtered['PassAtt'] / final_out_stats_filtered['90s']).round(2)
final_out_stats_filtered.loc[:, 'PrgPass/90'] = (final_out_stats_filtered['PrgPass'] / final_out_stats_filtered['90s']).round(2)
final_out_stats_filtered.loc[:, 'PrgPassRec/90'] = (final_out_stats_filtered['PrgPassRec'] / final_out_stats_filtered['90s']).round(2)
final_out_stats_filtered.loc[:, 'Tkl/90'] = (final_out_stats_filtered['Tkl'] / final_out_stats_filtered['90s']).round(2)
final_out_stats_filtered.loc[:, 'TklW/90'] = (final_out_stats_filtered['TklW'] / final_out_stats_filtered['90s']).round(2)
final_out_stats_filtered.loc[:, 'Int'] = (final_out_stats_filtered['Tkl+Int'] - final_out_stats_filtered['Tkl'])
final_out_stats_filtered.loc[:, 'Int/90'] = (final_out_stats_filtered['Int'] / final_out_stats_filtered['90s']).round(2)
final_out_stats_filtered.loc[:, 'Clr/90'] = (final_out_stats_filtered['Clr'] / final_out_stats_filtered['90s']).round(2)
final_out_stats_filtered.loc[:, 'Recov/90'] = (final_out_stats_filtered['Recov'] / final_out_stats_filtered['90s']).round(2)
final_out_stats_filtered.loc[:, 'Touches/90'] = (final_out_stats_filtered['Touches'] / final_out_stats_filtered['90s']).round(2)
final_out_stats_filtered.loc[:, 'PressSucc/90'] = (final_out_stats_filtered['PressSucc'] / final_out_stats_filtered['90s']).round(2)
final_out_stats_filtered.loc[:, 'ADuelWon/90'] = (final_out_stats_filtered['ADuelWon'] / final_out_stats_filtered['90s']).round(2)

In [16]:
# Columns to keep for the final output
columns_to_keep = [
    'ID', 'Player', 'Team', 'Age', 'Nationality', 'Season', 'MV1', 'MV2',
    'Position', 'Pos', 'Comp', 'MP', 'Starts', 'Min', '90s',
    'npG', 'npG/90', 'npG-npxG', 'Sh', 'SoT', 'SoT%', 'Sh/90', 'SoT/90',
    'SCA', 'GCA', 'SCA/90', 'GCA/90', 'DribSucc', 'DribSucc/90', 'Drib%',
    'PassCmp', 'PassCmp/90', 'PassAtt', 'PassAtt/90', 'Pass%', 'PrgPass', 'PrgPass/90', 'PrgPassRec', 'PrgPassRec/90',
    'Tkl', 'Tkl/90', 'TklW', 'TklW/90', 'Int', 'Int/90', 'Clr', 'Clr/90', 'Recov', 'Recov/90',
    'PressSucc', 'PressSucc/90', 'Press%', 'ADuelWon', 'ADuelWon/90', 'ADuel%',
    'Touches', 'Touches/90', 'PPM', 'On-Off']

# Filter the columns to keep
final_out_stats_filtered = final_out_stats_filtered[columns_to_keep]

In [17]:
# Create a copy of the DataFrame
final_gk_stats_filtered = final_merged_gk_stats.copy()

# Renaming columns
final_gk_stats_filtered.rename(columns={
    'player_id': 'ID',
    'Squad': 'Team',
    'Nationality Cleaned': 'Nationality',
    'Position Grouped': 'Position',
    'market_value_1': 'MV1',
    'market_value_2': 'MV2',
    'GA90': 'GA/90',
    'Save%.1': 'PKSave%',
    'PSxG+/-': 'PSxG-GA',
    '/90': 'PSxG-GA/90',
    'Opp': 'CrossAtt',
    'PKA': 'PKAtt',
    'Cmp%': 'Pass%',
    'Cmp%.3': 'LPass%',
    '#OPA': 'ActOutPA',
    '#OPA/90': 'ActOutPA/90',
    'Stp%': 'CrossStop%',
    'Att.2': 'SPassAtt',
    'Cmp.2': 'SPassCmp',
    'Cmp%.2': 'SPass%',
    'Att.3': 'LPassAtt',
    'Cmp.3': 'LPassCmp',
    'Cmp%.3': 'LPass%'
    },
                                inplace=True)

# Creating new columns using .loc
final_gk_stats_filtered.loc[:, 'SoTA/90'] = (final_gk_stats_filtered['SoTA'] / final_gk_stats_filtered['90s']).round(2)
final_gk_stats_filtered.loc[:, 'CrossAtt/90'] = (final_gk_stats_filtered['CrossAtt'] / final_gk_stats_filtered['90s']).round(2)
final_gk_stats_filtered.loc[:, 'PKAtt/90'] = (final_gk_stats_filtered['PKAtt'] / final_gk_stats_filtered['90s']).round(2)
final_gk_stats_filtered.loc[:, 'SPassAtt/90'] = (final_gk_stats_filtered['SPassAtt'] / final_gk_stats_filtered['90s']).round(2)
final_gk_stats_filtered.loc[:, 'LPassAtt/90'] = (final_gk_stats_filtered['LPassAtt'] / final_gk_stats_filtered['90s']).round(2)
final_gk_stats_filtered.loc[:, 'SPassCmp/90'] = (final_gk_stats_filtered['SPassCmp'] / final_gk_stats_filtered['90s']).round(2)
final_gk_stats_filtered.loc[:, 'LPassCmp/90'] = (final_gk_stats_filtered['LPassCmp'] / final_gk_stats_filtered['90s']).round(2)

In [18]:
# Columns to keep for the final output
gk_columns_to_keep = [
    'ID', 'Player', 'Team', 'Age', 'Nationality', 'Season', 'MV1', 'MV2', 'Position', 'Comp', 'MP', 'Starts', 'Min', '90s',
    'GA', 'GA/90', 'SoTA','SoTA/90', 'PSxG-GA', 'PSxG-GA/90', 'Save%',
    'CS%','CrossAtt', 'CrossAtt/90', 'CrossStop%','PKAtt', 'PKAtt/90', 'PKSave%',
    'ActOutPA', 'ActOutPA/90', 'SPassAtt', 'SPassAtt/90', 'SPassCmp', 'SPassCmp/90',
    'LPassAtt', 'LPassAtt/90', 'LPassCmp', 'LPassCmp/90', 'SPass%', 'LPass%', 'PPM', 'On-Off']

# Filter the columns to keep
final_gk_stats_filtered = final_gk_stats_filtered[gk_columns_to_keep]

In [20]:
final_gk_stats_filtered.to_csv('../data/processed/goalkeepers.csv', index=False)
final_out_stats_filtered.to_csv('../data/processed/outfielders.csv', index=False)
